Imports

In [ ]:
!pip install google-api-python-client google-auth-oauthlib

In [136]:
#imports
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import os
from googleapiclient import discovery
from base64 import urlsafe_b64decode, urlsafe_b64encode
from email.mime.text import MIMEText

In [108]:

def authenticate(SCOPES):
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'cred.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

In [140]:

def start_connect():
    SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
    creds = authenticate()
    return  discovery.build('gmail', 'v1', credentials=creds)
    
    

In [141]:
service=start_connect()

In [142]:
#return unreded emails in syntex [{'from':From,'Date':Date,'subject':subject,'message':mess}]
def read_emails():
    data=[]
    results = service.users().threads().list(userId='me',labelIds=['INBOX','UNREAD']).execute()
    threads = results.get('threads', [])

    for thread in threads:
        thread_info = service.users().threads().get(userId='me', id=thread['id']).execute()
        messages = thread_info['messages']
       
        for message in messages:
            headers=message['payload']['headers']
            From=""
            Date=""
            subject=""
            mess=""
            if message['payload']['body']['size'] >0:
                mess=message['payload']['body']['data']
                mess=urlsafe_b64decode(mess).decode('utf-8')
            for header in headers:
                if header['name']=='from':
                    From=header['name']
                if header['name']=='Date':
                    Date=header
                if header['name']=='Subject':
                    subject=header
            temp=dict({'from':From,
            'Date':Date,
            'subject':subject,
            'message':mess
            })
            if(len(subject)>0):
                data.append(temp)
                      
        for msg in messages:
            service.users().messages().modify(
            userId='me',
            id=msg['id'],
            body={'removeLabelIds': ['UNREAD']}).execute()
    return data
            

In [143]:

def create_draft(service, sender, to, subject, body_text):
    message = MIMEText(body_text)
    message["to"] = to
    message["from"] = sender
    message["subject"] = subject

    # base64url encode
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    # wrap in draft
    draft = {"message": {"raw": encoded_message}}

    # create draft
    draft = service.users().drafts().create(userId="me", body=draft).execute()
    print(f"Draft id: {draft['id']}")
    return draft


In [173]:
def get_draft(id):
    return service.users.drafts.get(userId="me", id=draft_id, format="full").execute()
    

In [152]:
def change_draft(service, sender, to, subject, body_text,id):
    message = MIMEText(body_text)
    message["to"] = to
    message["from"] = sender
    message["subject"] = subject

    # base64url encode
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    # wrap in draft
    draft = {"message": {"raw": encoded_message}}
    service.users().drafts().update(userId='me',id=id,body=encoded_message)

    

In [171]:
def draft_send(service,id):
    service.users().drafts().send(userId='me',body={"id":id}).execute()
    service.users().drafts().delete(userId='me',id=id).execute()
    